In [1]:
library(caret)
library(tidyverse)
library(caretEnsemble)
library(fastDummies)

Loading required package: ggplot2

Loading required package: lattice

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ purrr::lift()   masks caret::lift()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘caretEnsemble’


The following object is masked from ‘package:ggplot2’:

    autoplot


Thank you for using fastDummies!

To acknowledge our work, please cite the package:

Kaplan, J. & Schlegel, B. (2023). fastDummies: Fast Creation of Dummy (Binary) Columns and Rows from Categorical Variables. Version 1.7.1. URL: https://github.com/jacobkap/fastDummies, https://jaco

In [2]:
load('training.RData')
load('testing.RData')
training = training_imp
testing = testing_imp

In [4]:
#basic parameter tuning
fitControl <- trainControl(## 5-fold CV
  method = "cv", number = 5, verboseIter = TRUE,returnData = FALSE,trim = TRUE,
  savePredictions = "final")


In [3]:
training = dummy_cols(training, remove_first_dummy = TRUE, remove_selected_columns = TRUE)
testing = dummy_cols(testing, remove_first_dummy = TRUE, remove_selected_columns = TRUE)

In [ ]:
write.csv(training, 'training.csv', row.names = FALSE)
write.csv(testing, 'testing', row.names = FALSE)

In [11]:

model_list_complete <- caretList(
  x = training[,-21],
  y = training$UnitLengthStay,
  trControl = fitControl,
  metric = "RMSE",
  tuneList = list(
    lm = caretModelSpec(method = "lm"),
    gbm = caretModelSpec(method = "xgbTree")
    )
)

Warning message in trControlCheck(x = trControl, y = target):
“indexes not defined in trControl.  Attempting to set them ourselves, so each model in the ensemble will have the same resampling indexes.”


+ Fold1: intercept=TRUE 
- Fold1: intercept=TRUE 
+ Fold2: intercept=TRUE 
- Fold2: intercept=TRUE 
+ Fold3: intercept=TRUE 
- Fold3: intercept=TRUE 
+ Fold4: intercept=TRUE 
- Fold4: intercept=TRUE 
+ Fold5: intercept=TRUE 
- Fold5: intercept=TRUE 
Aggregating results
Fitting final model on full training set
+ Fold1: eta=0.3, max_depth=1, gamma=0, colsample_bytree=0.6, min_child_weight=1, subsample=0.50, nrounds=150 
[18:52:23] WARNING: src/c_api/c_api.cc:935: `ntree_limit` is deprecated, use `iteration_range` instead.
[18:52:23] WARNING: src/c_api/c_api.cc:935: `ntree_limit` is deprecated, use `iteration_range` instead.
- Fold1: eta=0.3, max_depth=1, gamma=0, colsample_bytree=0.6, min_child_weight=1, subsample=0.50, nrounds=150 
+ Fold1: eta=0.3, max_depth=1, gamma=0, colsample_bytree=0.6, min_child_weight=1, subsample=0.75, nrounds=150 
[18:52:36] WARNING: src/c_api/c_api.cc:935: `ntree_limit` is deprecated, use `iteration_range` instead.
[18:52:36] WARNING: src/c_api/c_api.cc:935: 

: 

In [ ]:
save(model_list_complete,file="model_list_complete.RData")


In [6]:
load('model_list_complete.RData')


In [8]:
stacking_model  <- caretStack(
  model_list_complete, 
  trControl=fitControl,
  metric="RMSE")

: 

In [ ]:

Observed = data.frame(Observed=testing$UnitLengthStay)
Predicted = data.frame(Predicted = predict(stacked_model,newdata=testing))

In [16]:
Erro = RMSE(pred =  Predicted$Predicted, obs = Observed$Observed)
MAE = MAE(pred = Predicted$Predicted, obs = Observed$Observed)
R2 = R2(pred = Predicted$Predicted, obs = Observed$Observed)

comparison = as.data.frame(cbind(Observed,Predicted))

ggplot(aes(x=Predicted,y=Observed),data=comparison)+ggtitle("Calibration for Updated Stacking model")+
  geom_smooth()+
  geom_segment(aes(x=0,y=0,xend=21,yend=21))+
  xlab("Predicted ICU length of stay")+
  ylab("Observed ICU length of stay")

ERROR: Error in RMSE(pred = Predicted$Predicted, obs = Observed$Observed): unused arguments (pred = Predicted$Predicted, obs = Observed$Observed)
